<a href="https://colab.research.google.com/github/kyulee-jeon/Tutorial/blob/main/KOMATE_unique_patient_oldest_ecg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime as datetime


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/ECG_AGE/KOMATE_Lesion_Based_V1.0.xlsx')

In [ ]:
df_scrap = pd.read_excel('/content/drive/MyDrive/ECG_AMI/SCRAP.xlsx')

In [61]:
df_scrap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20030 entries, 0 to 20029
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   CHART_NO   20030 non-null  int64
 1   AIsUnitNO  20030 non-null  int64
dtypes: int64(2)
memory usage: 313.1 KB


(0) 필요한 열만 불러와서 데이터 타입 확인하기

In [ ]:
df1 = df[['CHART_NO', 'LESION_DATE', 'STEMI_1']] #dataframe에서 특정 열 추출: df[['col name']]
df1.info()

'''
주의: 이때, dtype = object 를 유심히 봐야 함. 
알고보니 해당 데이터에는 다른 형식의 string data가 들어있어서
나중에 오류가 나왔음. 미리 확인해줘야한다.
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30166 entries, 0 to 30165
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CHART_NO     30166 non-null  object
 1   LESION_DATE  30166 non-null  object
 2   STEMI_1      30166 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 707.1+ KB


'\n주의: 이때, dtype = object 를 유심히 봐야 함. \n알고보니 해당 데이터에는 다른 형식의 string data가 들어있어서\n나중에 오류가 나왔음. 미리 확인해줘야한다.\n'

In [ ]:
# CHART_NO : weird 값 제거
df1['CHART_NO'] = df1['CHART_NO'].astype(str) # dataframe 데이터 타입 변경: df.astype() 
temp = df1['CHART_NO'].apply(lambda x: x.isdigit()) # 문자열.isdigit() : return True(숫자) or False
int_index = np.where(temp==True)
df1 = df1.loc[int_index] # index에 해당하는 행만 추출

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#import datetime as datetime
df1['LESION_DATE']=pd.to_datetime(df1['LESION_DATE']) #dataframe 날짜 형식 변환: pd.to_datetime(df['col name'])
df1['CHART_NO'] = df1['CHART_NO'].astype(int) 

In [57]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30161 entries, 0 to 30165
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   CHART_NO     30161 non-null  int64         
 1   LESION_DATE  30161 non-null  datetime64[ns]
 2   STEMI_1      30161 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 942.5 KB


(1) CHART_NO 별 가장 예전의 LESION_DATE만 가져오기 (df2)

In [62]:
# 30161 = 2729 + 27432
df_s1 = df1[df1['STEMI_1']==1] #2729
df_s0 = df1[df1['STEMI_1']==0] #27432

In [66]:
# CHART_NO 별 가장 예전 날짜만 추출(groupby)
df_s1 = df_s1.groupby('CHART_NO').min().reset_index() #2158 : unique
df_s0 = df_s0.groupby('CHART_NO').min().reset_index() #18379 : unique 

# total : 20537

(2) df_s1, df_s0 간의 중복 제거 (df_s1 살리기)

In [70]:
# df_s1과 df_s0 간에 CHART_NO가 같은 경우는, df_s1만 살리고 df_s0은 버린다.
# 결합 데이터프레임에서 중복된 행은 지우는데, STEMI_1==1인건 남기고 0만 지운다.
# 결합 데이터프레임에서 중복된 행이 뭔지 확인하기
df_min_duplicated = pd.concat([df_s1 , df_s0]).reset_index(drop=True) #20537

In [87]:
# 인덱스 붙이기
df_min_duplicated.reset_index(inplace=True)

In [88]:
dup=df_min_duplicated[df_min_duplicated['CHART_NO'].duplicated()] #407 (전부 STEMI_1==0)..고맙다 파이썬

In [91]:
# 중복행 인덱스 뽑아내기
dup_index=dup['index'].to_list()

In [94]:
#결합데이터(df_min_duplicated)에서 dup에 해당하는 행을 모두 제거하면 된다!
df_min = df_min_duplicated.drop(dup_index)

(3) table 합치기 (df_min + df_scrap) merge

In [95]:
df3 = pd.merge(df_min, df_scrap, how='left', on='CHART_NO')

In [97]:
df3 = df3.dropna(axis=0) #결측값 제거

In [102]:
df_komate = df3.drop(['CHART_NO', 'index'], axis=1) # 특정 열 삭제

In [103]:
df_komate.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20030 entries, 0 to 20121
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   LESION_DATE  20030 non-null  datetime64[ns]
 1   STEMI_1      20030 non-null  int64         
 2   AIsUnitNO    20030 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 625.9 KB


In [108]:
# 저장
df_komate.to_csv('drive/MyDrive/ECG_AMI/KOMATE_unique_patient_oldest_ecg.csv')

In [104]:
df_komate

,LESION_DATE,STEMI_1,AIsUnitNO
0,2008-03-01,1,1806094.0
1,2013-11-23,1,2887603.0
2,2012-05-29,1,3997298.0
3,2006-12-06,1,3544396.0
4,2010-06-18,1,3390278.0
...,...,...,...
20117,2022-02-21,0,10646723.0
20118,2022-02-24,0,10647947.0
20119,2022-02-23,0,10650951.0
20120,2022-02-24,0,10650635.0


In [105]:
df_komate['AIsUnitNO'].nunique()

20030

In [107]:
df_komate[df_komate['STEMI_1']==0]

,LESION_DATE,STEMI_1,AIsUnitNO
2158,2009-03-05,0,4535311.0
2159,2014-07-08,0,2161888.0
2160,2017-11-24,0,4222459.0
2161,2006-04-06,0,2795185.0
2162,2008-04-24,0,3126400.0
...,...,...,...
20117,2022-02-21,0,10646723.0
20118,2022-02-24,0,10647947.0
20119,2022-02-23,0,10650951.0
20120,2022-02-24,0,10650635.0


In [ ]:
17883+2147

20030